### Importing Library

In [1]:
# Your code here - remember to use markdown cells for comments as well!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
! cd ../..

### Importing data

In [2]:
tn_budgets = pd.read_csv("../../tn.movie_budgets.csv.gz")#../
bom = pd.read_csv('../../data/zippedData/bom.movie_gross.csv.gz')
rt_movie = pd.read_csv("../../data/zippedData/rt.movie_info.tsv.gz", sep='\t')
rt_reviews = pd.read_csv("../../data/zippedData/rt.reviews.tsv.gz", sep='\t', encoding='windows-1252')
tmdb = pd.read_csv("../../data/zippedData/tmdb.movies.csv.gz")
conn = sqlite3.connect('im.db')
cur = conn.cursor()
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
table_names = cur.fetchall()

FileNotFoundError: [Errno 2] No such file or directory: '../tn.movie_budgets.csv.gz'

### Redudant Data Cleaning

In [ ]:
tn_budgets['release_date'] = pd.to_datetime(tn_budgets['release_date'])
tn_budgets['production_budget'] = tn_budgets['production_budget'].str.replace('$','')
tn_budgets['production_budget'] = tn_budgets['production_budget'].str.replace(',','')
tn_budgets = tn_budgets.astype({'production_budget': 'int64'})
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].str.replace('$','')
tn_budgets['domestic_gross'] = tn_budgets['domestic_gross'].str.replace(',','')
tn_budgets = tn_budgets.astype({'domestic_gross': 'int64'})
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].str.replace('$','')
tn_budgets['worldwide_gross'] = tn_budgets['worldwide_gross'].str.replace(',','')
tn_budgets = tn_budgets.astype({'worldwide_gross': 'int64'})

### Cleaning Studio columns to remove duplicates

In [ ]:
bom['studio'].unique()
bom['studio'] = bom['studio'].str.strip('()')
bom['studio'] = bom['studio'].str.strip('(NL')
bom['studio'] = bom['studio'].str.strip()
bom['studio'] = bom['studio'].str.strip()
bom['studio'] = bom['studio'].str.replace('BV','Walt Disney')
bom['studio'] = bom['studio'].str.replace('P/DW','Pixar')
bom['studio'] = bom['studio'].str.replace('Uni.','Universial')
bom['studio'] = bom['studio'].str.replace('Par.','Paramount')
bom['studio'] = bom['studio'].str.strip()

### Feature Creation create year column & Join Merge Bom & Tn_budgets

In [ ]:
## Adding a year column
tn_budgets['year'] =  pd.DatetimeIndex(tn_budgets['release_date']).year
### Merging bom dataframe and Tn budgets
bom_budgets = pd.merge(tn_budgets, bom[['studio','title', 'year']],left_on=['movie','year'], right_on = ['title','year'], how = 'inner')
### Creating a column called net that calculates the difference between worldwide_gross and production budget
bom_budgets['net'] = bom_budgets['worldwide_gross'] - bom_budgets['production_budget']

### Groups by studio and find the top ten Studio by net profit

In [ ]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
net_studio = bom_budgets.groupby("studio")['net'].mean().to_frame(name = 'average_net').reset_index()
top10_studio = net_studio.sort_values(by=['average_net'], ascending= False, na_position='first').head(10)
#Reduces the amount of digits displayed
#top10_studio['average_net'] = top10_studio['average_net'] / 100

### Bargraph of  top ten studio by Average net revenue

In [ ]:
sns.set_style("whitegrid")
#setting figure size 
plt.figure(figsize =(9,7))
#creating bar plot
ax_studio = sns.barplot(data=top10_studio, x='studio', y='average_net',color='#990000' )
#setting title and axis names
ax_studio.set_title('Average Net Revenue by Studio', fontsize = 14, weight = 'bold')
ax_studio.set_ylabel('Average Net Revenue (hundred millions)', fontsize = 12, weight = 'bold' )
ax_studio.set_xlabel('Studio', fontsize = 12, weight = 'bold')
ax_studio.set_xticklabels(ax_studio.get_xticklabels(), rotation=75);
plt.show()

### Bar Graph of top ten studios whose production cost is in the 25th interquartile range

In [ ]:
#grouping by studios by production cost
studio_cost = bom_budgets.groupby("studio")['production_budget'].mean().to_frame(name = 'average_cost').reset_index()
#creating_quartiles to find what inter quartile ranges of cost
studio_cost_qt = studio_cost['average_cost'].quantile([0.25, 0.5, 0.75])
# Selecting only studios in the 25th quartile or lower 
low_budget_studios = studio_cost[studio_cost['average_cost'] <= 6407142.857]
list_low = list(low_budget_studios['studio'])
low_cost_studios = bom_budgets[bom_budgets['studio'] == 'Viv' ]
#looping through a the list of every studio that is in the 25th quartile and appending it to a newframe
for i in list_low:
    low_cost_studios = bom_budgets[bom_budgets['studio'] ==  i].append(low_cost_studios)

In [ ]:
sns.set_style("whitegrid")
#creating bar plot
ax_studio_low = sns.barplot(data=top10_low, x='studio', y='average_net')
#setting figure size 
plt.figure(figsize =(9,7))
#setting title and axis names 
ax_studio_low.set_title('Average Net Revenue by Studio', fontsize = 14, weight = 'bold')
ax_studio_low.set_ylabel('Average Net Revenue (hundred millions)', fontsize = 12, weight = 'bold' )
ax_studio_low.set_xlabel('Studio', fontsize = 12, weight = 'bold')
ax_studio_low.set_xticklabels(ax.get_xticklabels(), rotation=75);

plt.show()